In [1]:
library(dplyr)
library(purrr)
library(stringr)

library(cem)
library(MatchIt)
library(lmtest)
library(sandwich)
library(cobalt)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tcltk

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: lattice


How to use CEM? Type vignette("cem")


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


 cobalt (Version 4.4.0, Build Date: 2022-08-13)


Attaching package: ‘cobalt’


The following object is masked from ‘package:MatchIt’:

    lalonde




In [2]:
str_prod = function(x1, x2){
    out = c()
    for(elem1 in x1){
        for(elem2 in x2){
            out = append(out, paste0(elem1, elem2))
        }
    }
    return(out)
}

In [3]:
# Global Variables

location_category = c("HOME","WORK","OTHERS")
time_category = "TOD"

activity_category= "SEDENTARY"
dut_prefix = "DUT_"

app_category = c("ENTER","HEALTH","INFO","SOCIAL","SYSTEM","WORK")

# Feacture Extracted from each catergorical app usage
# UT: Usage Time, LC: Launch Count
app_prefixes = c("LC_", "UT_") 
app_usage = str_prod(app_prefixes, app_category)
all_app_usage = str_prod(app_prefixes,c('ALL'))

all_app_usage
app_usage

[1] "LC_ALL" "UT_ALL"

[1] "LC_ENTER"  "LC_HEALTH" "LC_INFO"   "LC_SOCIAL" "LC_SYSTEM" "LC_WORK"  
 [7] "UT_ENTER"  "UT_HEALTH" "UT_INFO"   "UT_SOCIAL" "UT_SYSTEM" "UT_WORK"

In [4]:
# file_name: file of raw data
# isCategory: Category or All
prepareData = function(df, isCategory){

    # Extract TOD
    time = str_split_fixed(df$READABLE_START, " ", 2)[,2]
    hour = as.numeric(str_split_fixed(time, ":", 2)[,1])
    df[time_category] = as.integer(hour/6)    
    
    # Construct Variable List used for Causal Inference
    variable_list = if(isCategory) app_usage else all_app_usage
    variable_list = append(variable_list, paste0(dut_prefix,location_category))
    variable_list = append(variable_list, paste0(dut_prefix,activity_category))
    variable_list = append(variable_list, time_category)
    df = df[,variable_list]

    # Remove TOD with 0
    df = df[df[time_category] != 0,]
    rownames(df) = NULL 
    df[,time_category] = as.factor(df[,time_category])

    # Min-Max Normalization
    df = as.data.frame(sapply(df, function(x){return(if(is.numeric(x)) (x-min(x))/(max(x)-min(x)) else x)}))
    # Remove NaN
    removed = colnames(df[,colSums(is.na(df)) != 0])
    df = df[,colSums(is.na(df)) == 0]

    out = list()
    out$df = df
    out$removed_col = removed

    return(out)
}

df=read.csv("Data/15Min/P1.csv")
out = prepareData(df, TRUE)
df_processed = out$df

summary(df_processed)

    LC_ENTER          LC_INFO          LC_SOCIAL         LC_SYSTEM      
 Min.   :0.00000   Min.   :0.00000   Min.   :0.00000   Min.   :0.00000  
 1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.00000  
 Median :0.00000   Median :0.00000   Median :0.00000   Median :0.03448  
 Mean   :0.01762   Mean   :0.01202   Mean   :0.06954   Mean   :0.11796  
 3rd Qu.:0.00000   3rd Qu.:0.00000   3rd Qu.:0.08929   3rd Qu.:0.17241  
 Max.   :1.00000   Max.   :1.00000   Max.   :1.00000   Max.   :1.00000  
    LC_WORK           UT_ENTER          UT_INFO          UT_SOCIAL      
 Min.   :0.00000   Min.   :0.00000   Min.   :0.00000   Min.   :0.00000  
 1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.00000   1st Qu.:0.00000  
 Median :0.00000   Median :0.00000   Median :0.00000   Median :0.00000  
 Mean   :0.01029   Mean   :0.02646   Mean   :0.01738   Mean   :0.09884  
 3rd Qu.:0.00000   3rd Qu.:0.00000   3rd Qu.:0.00000   3rd Qu.:0.09941  
 Max.   :1.00000   Max.   :1.00000   Max.   :1.0000

In [6]:
# Investigate the covariates based on the correlation with the outcome
# Calculate and return sig, nonsig list
sigCorr = function(df, outcome_variable){
    df = df[, sapply(df, is.numeric)]

    test = function (col1, col2){
        stat = cor.test(df[,col1], df[,col2], method = "kendall")
        return(stat$p.value < .05 && abs(stat$estimate['tau']) > .2)
    }
    sig_corr = c()
    for(col in colnames(df)){
        if(col != outcome_variable){
            if(test(outcome_variable, col)) sig_corr =append(sig_corr, col)
        } 
    }
    return(sig_corr)
}

df=read.csv("Data/15Min/P1.csv")
out = prepareData(df, TRUE)
df_processed = out$df

sigCorr(df_processed, "DUT_SEDENTARY")


[1] "LC_INFO"   "LC_SOCIAL" "LC_SYSTEM" "UT_INFO"   "UT_SOCIAL" "UT_SYSTEM"
[7] "DUT_HOME"

In [7]:
# Matching Process 
# formula: string as formula
# distance: ["PS", "Mahalanobis", "CEM"]
# method: ["nearest","full"] for "PS" and Mahalanobis

causal_inference = function(df, formula, outcome, distance, method = NULL, n_cutpoint_sampling = 10){
    formula = as.formula(formula)
    treat_variable = all.vars(formula)[1]
    confoundings = all.vars(formula)[-1]
    if(distance == "PS" || distance == "Mahalanobis"){
        m.out = matchit(formula, 
                        data= df,
                        method = if(distance == "PS") method else "nearest",
                        distance = if(distance == "PS") "glm" else "mahalanobis")
        m.balance = summary(m.out, un = FALSE)
        m.max_smd = max(abs(m.balance$sum.matched[-1,"Std. Mean Diff."]))
        m.treated = m.balance$nn["Matched","Treated"]
        m.data = match.data(m.out)
        m.att = NULL
        tryCatch({
            fit = lm(as.formula(paste(outcome, "~", treat_variable)), 
                data = m.data,
                weights = m.data$weights)
            m.att  = coeftest(fit, vcov. = vcovCL, cluster = m.data$subclass)
        }, error = function(e){
            stopifnot(m.balance$nn["Matched","Treated"] <= 1 || m.balance$nn["Matched","Control"] <= 1)
            # Error if treated or matched object exist for only once
            # print(e)
            # print(m.balance$nn)
            # print(formula)
        })

        
        out = list()
        out$m.out = m.out
        out$m.balance = m.balance
        out$m.max_smd = m.max_smd
        out$m.treated = m.treated
        out$m.att = m.att
        return(out)
    }else if(distance == "CEM"){
        drops = match(all.vars(formula), colnames(df))
        drops = colnames(df)[-drops]

        best.space = NULL
        best.treated = -1
        n_error = 0 
        best.bal.out = NULL

        for(i in 1:(n_cutpoint_sampling/10)){
            tryCatch({
                space = cemspace(treatment = treat_variable,
                        data = df,
                        drop = drops,
                        R = 10,
                        plot = FALSE,
                        minimal = 1,
                        maximal = 15)
                for(i in 1:10){
                    cem.out = cem(treat_variable, data = df, drop = drops, cutpoints =space$coars[[i]])
                    bal.out = bal.tab(cem.out, data = df, un = TRUE, stats= "m")
                    if(max(abs(bal.out$Balance$Diff.Adj))< .25 &&
                        cem.out$tab["Matched","G1"] > best.treated){
                        best.treated = cem.out$tab["Matched","G1"]
                        best.space = space$coars[[i]]
                    }
                }
            }, error = function(e){
                n_error = n_error + 1
            })
        }
        if(best.treated == -1){
            out = list()
            out$n_error = n_error
            return(out)
        }


        m.out = cem(treat_variable, data = df, drop = drops, cutpoints = best.space)
        m.balance = bal.tab(cem.out, data = df, un = TRUE, stats= "m")
        m.att = att(cem.out, as.formula(paste(outcome, treat_variable, sep= " ~ ")), data = df)

        out = list()
        out$m.out = m.out
        out$m.balance = m.balance
        out$m.att = m.att
        out$m.treated = best.treated
        out$n_error = n_error
        return(out)
    }else{
        stop("Undefined Distance")
    }
    # Assessing the balance
    
}
treatment = "UT_WORK"
outcome_variable = "DUT_SEDENTARY"

df = read.csv("Data/15Min/P2.csv")
out=prepareData(df, TRUE)
print(paste(out$removed, collapse = " "))
df = out$df

correlates = sigCorr(df, "DUT_SEDENTARY")
print(paste(correlates, collapse = " "))
variables = append(outcome_variable, correlates)
df = df[,variables]
idx = match(treatment, correlates)
if(!is.na(idx)){
    confoundings = correlates[-idx]
    df$treat = ifelse(df[,treatment]>mean(df[,treatment]), 1, 0)
    print(causal_inference(df,formula = paste("treat ~ ", paste(confoundings, collapse= " + "), sep = ""), 
                    outcome = outcome_variable, distance = "PS",method = "nearest"))
    print(causal_inference(df,formula = paste("treat ~ ", paste(confoundings, collapse= " + "), sep = ""), 
                    outcome = outcome_variable, distance = "PS",method = "full"))   
                                     
    print(causal_inference(df,formula = paste("treat ~ ", paste(confoundings, collapse= " + "), sep = ""), 
                    outcome = outcome_variable, distance = "Mahalanobis",method = "nearest"))

    print(causal_inference(df,formula = paste("treat ~ ", paste(confoundings, collapse= " + "), sep = ""), 
                    outcome = outcome_variable, distance = "CEM"))
}else{
    print("Treatment is not correlated with variables")
}


[1] "LC_HEALTH UT_HEALTH"
[1] "LC_SOCIAL LC_SYSTEM LC_WORK UT_SOCIAL UT_SYSTEM UT_WORK DUT_HOME"
$m.out
A matchit object
 - method: 1:1 nearest neighbor matching without replacement
 - distance: Propensity score
             - estimated with logistic regression
 - number of obs.: 390 (original), 46 (matched)
 - target estimand: ATT
 - covariates: LC_SOCIAL, LC_SYSTEM, LC_WORK, UT_SOCIAL, UT_SYSTEM, DUT_HOME

$m.balance

Call:
matchit(formula = formula, data = df, method = if (distance == 
    "PS") method else "nearest", distance = if (distance == "PS") "glm" else "mahalanobis")

Summary of Balance for Matched Data:
          Means Treated Means Control Std. Mean Diff. Var. Ratio eCDF Mean
distance         0.7149        0.2058          1.4136     2.1186    0.0541
LC_SOCIAL        0.2151        0.3501         -1.0235     0.2444    0.1465
LC_SYSTEM        0.3215        0.3860         -0.2807     0.8380    0.0819
LC_WORK          0.3120        0.0563          0.9300    17.8387    0.2870
U

In [8]:
individual = function(treatment, outcome, df, n_cutpoint_sampling){
    correlates = sigCorr(df, outcome)
    variables = append(outcome, correlates)
    idx = match(treatment, correlates)
    if(!is.na(idx))
        confoundings = correlates[-idx]
    else
        confoundings = correlates
    
    df$treat = ifelse(df[,treatment]>mean(df[,treatment]), 1, 0)

    formula =  paste("treat ~ ", paste(confoundings, collapse= " + "), sep = "")
    
    out = list()
    out$formula = formula    
    if(length(confoundings) > 0){
        out$ps.nearest = causal_inference(df,formula, outcome, distance = "PS",method = "nearest")
        out$ps.full = causal_inference(df,formula, outcome, distance = "PS",method = "full")
        out$mahalanobis = causal_inference(df,formula, outcome, distance = "Mahalanobis")
        out$cem = causal_inference(df, formula, outcome, distance = "CEM", n_cutpoint_sampling = n_cutpoint_sampling)
    }else{
        out$att = mean(df[which(df$treat ==  1),outcome]) - mean(df[which(df$treat == 0), outcome]) 
    }
    

    return(out)
}

datadir = 'Data/15Min'
result = list()
csv = "P3.csv"
df = read.csv(paste(datadir, csv, sep="/"))

out = list()
out$pid = strsplit(csv, split=".csv")[[1]]
prepared = prepareData(df,TRUE)
out$removed = prepared$removed
out$sig_corr = sigCorr(prepared$df, "DUT_SEDENTARY")

idx = match(out$removed, app_usage)
idx = idx[!is.na(idx)]
if(length(idx) > 0){
    left = app_usage[-idx]
}else{
    left = app_usage
}
for(elem in left){
    out[[paste("result",elem,sep=".")]] = individual(elem, "DUT_SEDENTARY", prepared$df, 10)
}
result[[out$pid]] = out


Calculating L1 profile for the raw data...


Executing 10 different random CEM solutions
  |                                                                      |   0%
Using 'treat'='1' as baseline group
  |========                                                              |  11%
Using 'treat'='1' as baseline group
  |================                                                      |  22%
Using 'treat'='1' as baseline group
  |=======================                                               |  33%
Using 'treat'='1' as baseline group
  |===============================                                       |  44%
Using 'treat'='1' as baseline group
  |=======================================                               |  56%
Using 'treat'='1' as baseline group
  |===============================================                       |  67%
Using 'treat'='1' as baseline group
  |======================================================                |  78%
Using 'treat'='1' 

In [9]:
print(result)

$P3
$P3$pid
[1] "P3"

$P3$removed
[1] "LC_HEALTH" "UT_HEALTH"

$P3$sig_corr
[1] "LC_INFO"   "LC_SOCIAL" "LC_SYSTEM" "UT_INFO"   "UT_SOCIAL" "UT_SYSTEM"
[7] "DUT_HOME" 

$P3$result.LC_ENTER
$P3$result.LC_ENTER$formula
[1] "treat ~ LC_INFO + LC_SOCIAL + LC_SYSTEM + UT_INFO + UT_SOCIAL + UT_SYSTEM + DUT_HOME"

$P3$result.LC_ENTER$ps.nearest
$P3$result.LC_ENTER$ps.nearest$m.out
A matchit object
 - method: 1:1 nearest neighbor matching without replacement
 - distance: Propensity score
             - estimated with logistic regression
 - number of obs.: 457 (original), 112 (matched)
 - target estimand: ATT
 - covariates: LC_INFO, LC_SOCIAL, LC_SYSTEM, UT_INFO, UT_SOCIAL, UT_SYSTEM, DUT_HOME

$P3$result.LC_ENTER$ps.nearest$m.balance

Call:
matchit(formula = formula, data = df, method = if (distance == 
    "PS") method else "nearest", distance = if (distance == "PS") "glm" else "mahalanobis")

Summary of Balance for Matched Data:
          Means Treated Means Control Std. Mean Diff. Var. Rati

In [10]:
#OVERALL APP USAGE
csv = "P4.csv"
out = list()
out$pid = strsplit(csv, split=".csv")[[1]]
prepared = prepareData(df,FALSE)
out$removed = prepared$removed
out$sig_corr = sigCorr(prepared$df, "DUT_SEDENTARY")

idx = match(out$removed, all_app_usage)
idx = idx[!is.na(idx)]
if(length(idx) > 0){
    left = all_app_usage[-idx]
}else{
    left = all_app_usage
}
for(elem in left){
    out[[paste("result",elem,sep=".")]] = individual(elem, "DUT_SEDENTARY", prepared$df, 10)
}

Warning message in max(abs(m.balance$sum.matched[-1, "Std. Mean Diff."])):
“no non-missing arguments to max; returning -Inf”



Calculating L1 profile for the raw data...


Executing 10 different random CEM solutions
  |                                                                      |   0%
Using 'treat'='1' as baseline group

Calculating L1 profile for the raw data...


Executing 10 different random CEM solutions
  |                                                                      |   0%
Using 'treat'='1' as baseline group
  |========                                                              |  11%
Using 'treat'='1' as baseline group
  |================                                                      |  22%
Using 'treat'='1' as baseline group
  |=======================                                               |  33%
Using 'treat'='1' as baseline group
  |===============================                                       |  44%
Using 'treat'='1' as baseline group
  |=======================================                               |  56%
Using 'treat'='1' as baseline group
  |=====

In [11]:
print(out)

$pid
[1] "P4"

$removed
character(0)

$sig_corr
[1] "LC_ALL"   "DUT_HOME"

$result.LC_ALL
$result.LC_ALL$formula
[1] "treat ~ DUT_HOME"

$result.LC_ALL$ps.nearest
$result.LC_ALL$ps.nearest$m.out
A matchit object
 - method: 1:1 nearest neighbor matching without replacement
 - distance: Propensity score
             - estimated with logistic regression
 - number of obs.: 457 (original), 270 (matched)
 - target estimand: ATT
 - covariates: DUT_HOME

$result.LC_ALL$ps.nearest$m.balance

Call:
matchit(formula = formula, data = df, method = if (distance == 
    "PS") method else "nearest", distance = if (distance == "PS") "glm" else "mahalanobis")

Summary of Balance for Matched Data:
         Means Treated Means Control Std. Mean Diff. Var. Ratio eCDF Mean
distance        0.3180        0.3178          0.0036     0.9942     0.002
DUT_HOME        0.1891        0.1905         -0.0037     0.9932     0.002
         eCDF Max Std. Pair Dist.
distance   0.0074          0.0059
DUT_HOME   0.0074     